In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, RMSprop
import time
%load_ext tensorboard

In [ ]:
from tensorflow.keras.callbacks import TensorBoard
Name = "Final_Pottery_Resnet_50_dropout0.2".format(int(time.time()))
path_dir ='/content/drive/MyDrive/restof_of_the_path_to_this_file'

tensorboard = TensorBoard(log_dir = '/content/drive/MyDrive/rest_of_the_path/logs/{}'.format(Name))


In [ ]:
import pathlib
data_dir = "/content/drive/MyDrive/_rest_of_the_path_to_this_file"
print(data_dir)
data_dir = pathlib.Path(data_dir)

In [ ]:
BATCH_SIZE = 12
SEED = 1
img_height,img_width = 600,720

train_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    label_mode = "binary",
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    image_size=(img_height, img_width),
    shuffle=True,
    validation_split=0.2,
    subset='training',
    seed=SEED)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    label_mode = "binary",
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    image_size=(img_height, img_width),
    shuffle=False,
    validation_split=0.2,
    subset='validation',
    seed=SEED)


In [ ]:
class_names = train_dataset.class_names
print(class_names)

In [ ]:
from tensorflow.image import flip_left_right, flip_up_down, rot90, grayscale_to_rgb, rgb_to_grayscale

def rescale_and_rgb(image, label):
    image = tf.cast(image, tf.float32)
    imge = (image/255.0)
    image = grayscale_to_rgb(image)
    return image, label

train_dataset = train_dataset.map(rescale_and_rgb, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(rescale_and_rgb, num_parallel_calls=tf.data.AUTOTUNE)

data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])

def prepare(ds, augment=False):
    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y),
                num_parallel_calls=tf.data.AUTOTUNE)
    # Use buffered prefetching on all datasets.
    return ds.prefetch(buffer_size=tf.data.AUTOTUNE)

train_dataset = prepare(train_dataset, augment=True)
val_dataset = prepare(val_dataset)
print("complete")

In [ ]:
from tensorflow.keras.applications import ResNet50
pretrained_model = ResNet50(
    include_top=False,
    weights="imagenet",
    input_shape =(600, 720, 3),
    classes = 2)

pretrained_model.summary()

In [ ]:
for layer in pretrained_model.layers:
  layer.trainable = False

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

costumized_resnet_model = Sequential()

costumized_resnet_model.add(pretrained_model)
costumized_resnet_model.add(GlobalAveragePooling2D())
costumized_resnet_model.add(Flatten())
costumized_resnet_model.add(Dropout(0.2))
costumized_resnet_model.add(Dense(512, "relu"))
costumized_resnet_model.add(Dense(1, "sigmoid"))

costumized_resnet_model.summary()

In [ ]:
from tensorflow.keras.metrics import *

metrics = ['accuracy', Precision(), Recall(), AUC()] # add F1Score() when not using tensorboard

from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.losses import BinaryCrossentropy

costumized_resnet_model.compile(loss=BinaryCrossentropy(),
                 optimizer=Adam(learning_rate=0.001),
                 metrics=metrics)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint


model_save_path = "/content/saved_models/{}".format(Name)

check_point = ModelCheckpoint( filepath=model_save_path, monitor= 'val_accuracy', mode = 'max', save_best_only=True,verbose=1)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=7, monitor='val_loss')

In [ ]:
epochs = 100
Resnet50_model_history = costumized_resnet_model.fit(train_dataset, epochs=epochs,
             validation_data=val_dataset, callbacks=[tensorboard,check_point])
print("complete")

In [ ]:



%tensorboard --logdir /content/drive/MyDrive/_rest_of_the_path_to_the_images
